Q1.3. Which are the top 10 pizza restaurants by rating?

In [2]:
# Import required libraries
import sqlite3
import pandas as pd

databases = ['../databases/deliveroo.db', '../databases/takeaway.db', '../databases/ubereats.db']

# Connect to the first database
conn = sqlite3.connect(databases[0])
cursor = conn.cursor()

# Dynamically attach other databases
for i, db in enumerate(databases[1:], start=2):
    cursor.execute(f"ATTACH DATABASE '{db}' AS db{i}")

In [3]:
query1 = """SELECT DISTINCT id, name, rating FROM (SELECT DISTINCT id, name, category, rating 
            FROM restaurants 
            WHERE category LIKE '%Pizza%' OR
            category LIKE '%pizza%'
             
            UNION
            SELECT DISTINCT restaurants.id AS id, restaurants.title AS name,  categories.category, restaurants.rating__rating_value AS rating
            FROM db3.restaurants 
            JOIN db3.restaurant_to_categories AS categories ON restaurants.id == categories.restaurant_id
            WHERE categories.category LIKE '%Pizza%' OR categories.category LIKE '%pizza%'

            UNION
            SELECT DISTINCT restaurants.primarySlug AS id, restaurants.name, categories.name, restaurants.ratings AS rating 
            FROM db2.categories
            JOIN db2.categories_restaurants ON categories.restaurant_id == categories_restaurants.restaurant_id
            JOIN db2.restaurants ON categories.restaurant_id == restaurants.primarySlug
            WHERE categories.name LIKE '%Pizza%' OR
            categories.name LIKE '%pizza%')
            ORDER BY rating DESC LIMIT 10
            ;
            """

df_pizza = pd.read_sql_query(query1, conn)
df_pizza#[['name', 'rating']]

,id,name,rating
0,eethuis-carlos-aarschot,De Echte Eethuis Carlos,5.0
1,kebap-pizza-de-brug,Kebap - Pizza De Brug,5.0
2,la-trca,La Trinacria Bottega,5.0
3,munchies-ieper,Munchies Ieper,5.0
4,snack-it,Snack-It,5.0
5,trend-kebab-pizzeria,Trend Kebab & Pizzeria,5.0
6,17478ff5-ba5d-5166-ba01-48d1f97cb588,American pizza,4.9
7,5a603fd8-4bfa-44e7-a4ad-47063893957f,Fratello's,4.9
8,am-princesse,Am Princesse,4.9
9,b0bc113f-86e9-41ed-95e1-7e8d4957ac9f,Pizza Autentica,4.9


In [4]:
import plotly.express as px
fig = px.bar(df_pizza, x='name', y="rating", color ="rating",
               labels={"name": "Name of restaurants serving pizza", "rating": "Rating for the restaurant"}, 
              title = "Top 10 pizza restaurants by rating")
fig.show()

Q2.3. Where are the delivery ‘dead zones’—areas with minimal restaurant coverage?

In [63]:
query2 = """SELECT count_of_restaurant, postal_code AS postcode, latitude, longitude FROM (SELECT DISTINCT COUNT(name) AS count_of_restaurant, postal_code,
AVG(latitude) AS latitude, AVG(longitude) AS longitude
FROM restaurants
WHERE postal_code IS NOT NULL
GROUP BY postal_code
UNION

SELECT DISTINCT COUNT(restaurants.name) AS count_of_restaurant, locations.postalCode AS postal_code,
AVG(locations.latitude) AS latitude, AVG(locations.longitude) AS longitude
FROM db2.restaurants
JOIN db2.locations_to_restaurants ON restaurants.primarySlug == locations_to_restaurants.restaurant_id
JOIN db2.locations ON locations_to_restaurants.location_id == locations.ID
WHERE postal_code IS NOT NULL
GROUP BY postal_code
UNION
SELECT DISTINCT COUNT(restaurants.title) AS count_of_restaurant,
       restaurants.location__postal_code as postal_code,
       AVG(restaurants.location__latitude) as latitude, 
       AVG(restaurants.location__longitude) as longitude
FROM db3.restaurants
WHERE postal_code IS NOT NULL
GROUP BY postal_code)
WHERE count_of_restaurant < 50
ORDER BY count_of_restaurant

"""

df_deadzone = pd.read_sql_query(query2, conn)
#df_deadzone = df_deadzone[df_deadzone['postcode'].str.isnumeric() & df_deadzone['postcode'].notna()]
df_deadzone.to_csv('dead_zone.csv')

In [72]:
# Function to color markers based on restaurant count
import folium
from folium.plugins import HeatMap
from matplotlib import colormaps
from matplotlib.colors import to_hex
import geopandas as gpd

map_dead_zones = folium.Map(location=[50.8503, 4.3517], zoom_start=8)
min = df_deadzone['count_of_restaurant'].min()
average_half = (df_deadzone['count_of_restaurant'].mean())/2
df_deadzone_min = df_deadzone[df_deadzone['count_of_restaurant'] == min]
def get_marker_color(count):
    if count == min:
        return 'red'
    elif count <= average_half:
        return 'orange'
    else:
        return 'green'

# Add all points to the map with color-coded markers
for _, row in df_deadzone_min.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Postal Code: {row['postcode']}, Restaurants: {row['count_of_restaurant']}",
        icon=folium.Icon(color=get_marker_color(row['count_of_restaurant'])),
    ).add_to(map_dead_zones)

map_dead_zones.save('../assets/dead_zones_map1.html')
map_dead_zones


In [71]:


import folium
from folium.plugins import HeatMap
from matplotlib import colormaps
from matplotlib.colors import to_hex
import geopandas as gpd

df = df_deadzone[df_deadzone['count_of_restaurant'] <= 50]


# Create a base map
#m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
m = folium.Map(location=[50.8503, 4.3517], zoom_start=8)  # Centered on Belgium

min_count = df['count_of_restaurant'].min()
max_count = df['count_of_restaurant'].max()

df['color'] = df['count_of_restaurant'].apply(lambda x: (x - min_count) / (max_count - min_count))
#df['radius'] = df['count_of_restaurant'].apply(lambda x: (max_count - x) / (max_count - min_count) * 15 + 5)
# Add markers to the map
#cmap = colormaps['RdYlGn'].reversed()
for _, row in df.iterrows():
    color = 'red' #if row['count_of_restaurant'] == 0 else 'green'
    #color = to_hex(cmap(row['color']))  # Convert RGBA to HEX
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius= 3, #row['radius'], #+ 3,  # Add small radius for visibility
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"postal_code: {row['postcode']}<br>count: {row['count_of_restaurant']}"
    ).add_to(m)

# Save or display the map
m.save('../assets/dead_zones_map2.html')
m

Q2.6. Identify the World Hummus Order (WHO); top 3 hummus serving restaurants.

In [67]:
query3 = """SELECT DISTINCT id, name, rating FROM (SELECT DISTINCT restaurants.id, restaurants.name, menu_items.name as menu, restaurants.rating 
            FROM restaurants
            JOIN menu_items ON restaurants.id == menu_items.restaurant_id
            WHERE menu_items.name LIKE '%Hummus%'
             
            UNION
            SELECT DISTINCT restaurants.primarySlug AS id, restaurants.name, menuItems.name, restaurants.ratings AS rating 
            FROM db2.menuItems
            JOIN db2.restaurants ON menuItems.primarySlug == restaurants.primarySlug
            WHERE menuItems.name LIKE '%Hummus%'

            UNION
            SELECT DISTINCT restaurants.id AS id, restaurants.title AS name,  menu_items.name, restaurants.rating__rating_value AS rating
            FROM db3.restaurants 
            JOIN db3.menu_items ON restaurants.id == menu_items.restaurant_id
            WHERE menu_items.name LIKE '%Hummus%')
            WHERE rating IS NOT NULL
            ORDER BY rating DESC
            LIMIT 5
            ;
            """

df_hummus = pd.read_sql_query(query3, conn)
df_hummus[['name', 'rating']]

,name,rating
0,D'Ostendsche Baguette,5.0
1,Punto Caffè,5.0
2,Restaurant le Royal,5.0
3,Royal Donuts & Bagels,5.0
4,Tabouleh,4.9


In [68]:
import plotly.express as px
fig = px.bar(df_hummus, x='name', y="rating", color='rating',
               labels={"name": "Name of restaurants", "rating": "Rating for the restaurant"}, 
              title = "Top 5 Hummus serving restaurants")
fig.show()

In [69]:
# Detach databases
for i in range(2, len(databases) + 1):
    cursor.execute(f"DETACH DATABASE db{i}")

# Close connection
conn.close()